# LAB 3: How to setup a project from Scratch

In [1]:
!rm -rf speech-emotion-recognition-25

# Step 1: Clone your project from Github

In [2]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch

!git clone -b IemocapClass https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 195 (delta 71), reused 58 (delta 32), pack-reused 94 (from 3)
Receiving objects: 100% (195/195), 70.29 KiB | 7.81 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [3]:
!ls

sample_data  speech-emotion-recognition-25


In [4]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [5]:
!ls

checkpoints  dataset  models	 requirements.txt		  train.py
data	     eval.py  README.md  SpeechEmotionRecnognition.ipynb  utils


# Step 2: Packages Installation


In [6]:
%pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [7]:
!python utils/download_dataset.py

--- 1. Configurazione Kaggle ---
Kaggle configurato con successo.

--- Download RAVDESS ---
Contatto KaggleHub per scaricare: uwrfkaggler/ravdess-emotional-speech-audio...
Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/ravdess-emotional-speech-audio
RAVDESS pronto in cache: /kaggle/input/ravdess-emotional-speech-audio
Numero totale di file: 2880

--- Download IEMOCAP ---
Contatto KaggleHub per scaricare: dejolilandry/iemocapfullrelease...
Using Colab cache for faster access to the 'iemocapfullrelease' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/iemocapfullrelease
IEMOCAP pronto in cache: /kaggle/input/iemocapfullrelease
Numero totale di file: 81249

RIEPILOGO DOWNLOAD
RAVDESS: ✅ Successo
IEMOCAP: ✅ Successo

🎉 Tutti i dataset sono stati scaricati con successo!


# Step 3.5: Improvised Vocal Recording

Model IEMOCAP dataset to extract only improvised audio. 

In [8]:
import os
from pathlib import Path

# Check kagglehub cache directory
kaggle_cache_base = Path('/root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease')

print("=" * 80)
print("KAGGLEHUB CACHE EXPLORATION")
print("=" * 80)

if kaggle_cache_base.exists():
    print(f"✓ Base cache directory exists: {kaggle_cache_base}\n")
    
    # Check versions subdirectory
    versions_dir = kaggle_cache_base / 'versions'
    if versions_dir.exists():
        print(f"✓ Versions directory exists\n")
        
        for version in sorted(versions_dir.iterdir()):
            if version.is_dir():
                print(f"📂 Version: {version.name}/")
                
                # Show IEMOCAP_full_release
                iemocap_dir = version / 'IEMOCAP_full_release'
                if iemocap_dir.exists():
                    print(f"  └─ 📂 IEMOCAP_full_release/")
                    
                    # Show Sessions
                    sessions = sorted([d for d in iemocap_dir.iterdir() if d.is_dir() and d.name.startswith('Session')])
                    print(f"    └─ Found {len(sessions)} sessions:")
                    
                    for session_dir in sessions[:2]:  # Show first 2 sessions
                        print(f"      📁 {session_dir.name}/")
                        
                        # Show subdirectories in session
                        subdirs = sorted([d.name for d in session_dir.iterdir() if d.is_dir()])
                        for subdir_name in subdirs[:5]:  # Show first 5 subdirs
                            print(f"        ├─ {subdir_name}/")
                        
                        if len(subdirs) > 5:
                            print(f"        └─ ... and {len(subdirs) - 5} more")
                    
                    if len(sessions) > 2:
                        print(f"      ... and {len(sessions) - 2} more sessions")
else:
    print(f"✗ Cache directory not found: {kaggle_cache_base}")

print("\n" + "=" * 80)


KAGGLEHUB CACHE EXPLORATION
✓ Base cache directory exists: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease

✓ Versions directory exists

📂 Version: 1/
  └─ 📂 IEMOCAP_full_release/
    └─ Found 5 sessions:
      📁 Session1/
        ├─ dialog/
        ├─ sentences/
      📁 Session2/
        ├─ dialog/
        ├─ sentences/
      ... and 3 more sessions



In [ ]:
from torch.utils.data import DataLoader
from dataset.custom_iemocap_dataset import CustomIEMOCAPDataset, collate_fn_iemocap

# Dataset path
dataset_path = '/root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release'

# Create datasets
train_dataset = CustomIEMOCAPDataset(dataset_root=dataset_path, split='train')
test_dataset = CustomIEMOCAPDataset(dataset_root=dataset_path, split='test')

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Create DataLoaders
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn_iemocap)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_iemocap)

✓ Found IEMOCAP in kagglehub cache
📂 Using dataset path: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release

🔄 Loading IEMOCAP dataset...
📂 Starting to collect samples from: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release
📂 Found 6 items in directory
   - Session5 (dir)
   - Session1 (dir)
   - Session3 (dir)
   - Session4 (dir)
   - Session2 (dir)
   ... and 1 more items
   Checking folder: Documentation
   Checking folder: Session1

📁 Processing Session1 (Session ID: 1)
   └─ Looking for audio in: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release/Session1/sentences/wav
   └─ Found impro folder: Ses01F_impro04
      Looking for label file: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro04.txt
      ✓ Ses01F_impro04_M036 → Label: xxx
   └─ Found impro folder: 

# Step 4: Train your model and visualize training

In [ ]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [ ]:
#!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [ ]:
#!python eval.py